In [1]:
# Import libraries

import sys
import pypsa
import logging
import pandas as pd
import numpy as np
import xarray as xr
from plotly import graph_objects as go
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.demand import projected_energy
from library.weather import load_weather

logging.basicConfig(level=logging.INFO)

In [2]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = True
use_h2 = True
h2_initial = 1000
biogas_limit = 500
load_target = 15

In [3]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [4]:
# Build the network

def annuity(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

def annualized_capex(asset):
    return (annuity(float(assumptions.loc[('general', 'discount_rate'), 'value']), float(assumptions.loc[(asset, 'lifetime'), 'value'])) + float(assumptions.loc[(asset, 'FOM'), 'value'])) * float(assumptions.loc[(asset, 'capital_cost'), 'value'])

## Initialize the network
network = pypsa.Network()
network.set_snapshots(index)
network.snapshot_weightings.loc[:, :] = resolution

## Carriers
carriers = [
    'AC',
    'onwind',
    'offwind',
    'solar',
    'li-ion',
    'h2',
    'biogas',
    'mixedgas',
    'backstop',
    ]

carrier_colors = ['black', 'green', 'blue', 'red', 'lightblue', 'grey', 'brown', 'brown', 'white']

network.madd(
    'Carrier',
    carriers,
    color=carrier_colors,
    )

## Load bus location
minx, miny, maxx, maxy = selection.total_bounds
midx = (minx + maxx)/2
midy = (miny + maxy)/2

## Add the buses
network.add('Bus', 'load-bus', carrier='AC', x=midx, y=midy)
network.add('Bus', 'renewables-bus', x=midx+0.5, y=midy+0.25)
network.add('Bus', 'battery-bus', carrier='li-ion', x=midx-0.5, y=midy)
if use_h2 or biogas_limit > 0:
    network.add('Bus', 'turbine-bus', x=midx, y=midy+0.5)
if use_h2:
    network.add('Bus', 'h2-bus', carrier='h2', x=midx-0.5, y=midy+0.5)
if biogas_limit > 0:
    network.add('Bus', 'biogas-bus', x=midx, y=midy+0.9)


## Add load and backstop to load bus
network.add('Load', 'load', bus='load-bus',
            p_set=target_load
            )

network.add('Generator', 'backstop', carrier='backstop', bus='load-bus',
            p_nom_extendable=True,
            capital_cost=assumptions.loc[('backstop', 'capital_cost'), 'value'],
            marginal_cost=assumptions.loc[('backstop', 'marginal_cost'), 'value'],
            lifetime=assumptions.loc[('backstop', 'lifetime'), 'value'],
            )

## Add generators and links to renewable bus

network.add('Generator', 'solar', carrier='solar', bus='renewables-bus',
            p_nom_extendable=True, 
            p_max_pu=capacity_factor_solar,
            p_nom_mod=assumptions.loc['solar','unit_size'].value,
            capital_cost= annualized_capex('solar'),
            marginal_cost=400,
            lifetime=assumptions.loc[('solar', 'lifetime'), 'value'],
            )

network.add('Generator', 'onwind', carrier='onwind', bus='renewables-bus',
            p_nom_extendable=True,
            p_max_pu=capacity_factor_onwind,
            p_nom_mod=assumptions.loc['onwind','unit_size'].value,
            capital_cost= annualized_capex('onwind'),
            marginal_cost=assumptions.loc[('onwind', 'VOM'), 'value'],
            lifetime=assumptions.loc['onwind','lifetime'].value,
            )

if use_offwind:
    network.add('Generator', 'offwind', carrier='offwind', bus='renewables-bus',
                p_nom_extendable=use_offwind,
                p_max_pu=(capacity_factor_offwind if use_offwind else [0] * len(capacity_factor_offwind)),
                p_nom_mod=assumptions.loc['offwind','unit_size'].value,
                capital_cost= annualized_capex('offwind'),
                marginal_cost=assumptions.loc[('offwind', 'VOM'), 'value'],
                lifetime=assumptions.loc['offwind','lifetime'].value,
                )

network.add('Link', 'Renewables load link', bus0='renewables-bus', bus1='load-bus',
            p_nom_extendable=use_offwind,
            )

## Add battery storage

network.add('Link','battery-charge', bus0='renewables-bus', bus1='battery-bus',
            p_nom_extendable = True,
            capital_cost= annualized_capex('battery_inverter'),
            marginal_cost=assumptions.loc['battery_inverter','VOM'].value,
            lifetime=assumptions.loc['battery_inverter','lifetime'].value,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

network.add('Store', 'battery', carrier='li-ion', bus='battery-bus',
            e_initial=100,
            e_nom_extendable=True,
            e_cyclic=True,
            e_min_pu=0.15,
            standing_loss=0.00008, # TODO: Check if this is really per hour as in the documentation or if it is per snapshot
            capital_cost= annualized_capex('battery_storage'),
            marginal_cost=assumptions.loc['battery_storage','VOM'].value*100,
            lifetime=assumptions.loc['battery_storage', 'lifetime'].value,
            )

network.add('Link','battery-discharge', carrier='li-ion', bus0='battery-bus', bus1='load-bus',
            p_nom_extendable = True,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

## Run optimization
network.optimize.solve_model(solver_name='highs')

{'biogas-bus', 'turbine-bus', 'h2-bus'}
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 1/1 [00:00<00:00, 982.04it/s]
INFO:linopy.io: Writing time: 0.28s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 26291 primals, 58411 duals
Objective: 2.55e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


  Matrix [4e-06, 2e+01]
  Cost   [9e+01, 2e+06]
  Bound  [0e+00, 0e+00]
  RHS    [1e+03, 3e+03]
Presolving model
27744 rows, 24829 cols, 71552 nonzeros  0s
23199 rows, 20284 cols, 70070 nonzeros  0s
23199 rows, 20284 cols, 70070 nonzeros  0s

Solving MIP model with:
   23199 rows
   20284 cols (0 binary, 3 integer, 0 implied int., 20281 continuous)
   70070 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.1s
 S       0       0         0   0.00%   -inf            25489494518.66     Large        0      0      0         0     1.1s
 R       0       0         0   0.00%   25486402634.62  25486427729.08     0.00%        0      0      0     24355     1.2s

Solving report
  S

('ok', 'optimal')

In [6]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator backstop       2525.386524                 0.0  6.371176e+05   
          onwind         7039.748571                 0.0  1.280402e+07   
          solar          6330.801960                 0.0  1.725990e+06   
Link      AC             4613.016848                 0.0  1.445088e+07   
          li-ion         2265.834579                 0.0  1.770766e+06   
Load      -                 0.000000                 0.0  0.000000e+00   
Store     li-ion        23814.762218                 0.0  1.846152e+06   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator backstop  0.000000e+00  6.371176e+05  0.000000e+00         0.028800   
          onwind    0.000000e+00  1.280402e+07  0.000000e+00         0.207628   
          solar     0.000000e+00  1.725990e+06  0.000000e+00         0.031123   
Link      AC        1.453001e+07 -7.913188e+04  1.259212e+07         0.359564   
          li-ion    1.846152e+06 -7.538578e+04  0.000000e+00         0.093011   
Load      -         1.500000e+07 -1.500000e+07  0.000000e+00              NaN   
Store     li-ion    1.858760e+06 -1.260800e+04  0.000000e+00         0.755511   

                     Curtailment  Capital Expenditure  \
Generator backstop  0.000000e+00         0.000000e+00   
          onwind    1.079892e+07         9.489303e+09   
          solar     5.425737e+06         3.288383e+09   
Link      AC        0.000000e+00         4.013085e+08   
          li-ion    0.000000e+00         0.000000e+00   
Load      -         0.000000e+00         0.000000e+00   
Store     li-ion    0.000000e+00         4.586533e+09   

                    Operational Expenditure  Revenue  Market Value  
Generator backstop             6.697380e+09      0.0           NaN  
          onwind               3.684942e+08      0.0           NaN  
          solar                6.903960e+08      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     li-ion              -3.536985e+07      0.0           NaN

In [7]:
network.buses_t.marginal_price.max()

Bus
load-bus          0.0
renewables-bus    0.0
battery-bus       0.0
turbine-bus       0.0
h2-bus            0.0
biogas-bus        0.0
dtype: float64